In [1]:
import os
import glob
import xml
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from tqdm.notebook import tqdm

In [2]:
# 漢数字を算用数字に直す関数
def kanji_to_number(kanji):
    if kanji:
        kanji_map = {
            '〇': 0, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
            '六': 6, '七': 7, '八': 8, '九': 9, '十': 10, '百': 100,
            '千': 1000, '万': 10000, '億': 100000000, '兆': 1000000000000,
        }
        result = 0
        temp = 0
        temp_10000 = 0
        current_multiplier = 1
        base_multiplier = 1

        for char in reversed(kanji):
            value = kanji_map[char]

            if value >= 10000:
                if current_multiplier > 1:
                    temp_10000 += current_multiplier
                temp += temp_10000 * base_multiplier
                base_multiplier = value
                current_multiplier = 1
                temp_10000 = 0
            elif value >= 10:
                if current_multiplier > 1:
                    temp_10000 += current_multiplier
                current_multiplier = value
            else:
                temp_10000 += value * current_multiplier
                current_multiplier = 1
        
        if current_multiplier > 1:
            temp_10000 += current_multiplier
        temp += temp_10000 * base_multiplier
        return str(temp)
    return ''

In [3]:
def wareki_to_seireki8(wareki):
    '''
    和暦から西暦8桁の数値を返す関数。
    warekiのテキストの中に複数の日付と思われるテキストとマッチした時には先頭の日付のみを返す。
    '''
    gengo_dict = {
        '明治':1867,
        '大正':1911,
        '昭和':1925,
        '平成':1988,
        '令和':2018,
    }
    pattern = re.compile(r'(明治|大正|昭和|平成|令和)([一二三四五六七八九十]+)年([一二三四五六七八九十]+)月([一二三四五六七八九十]+)日')
    wareki = wareki.replace('元年', '一年')
    date = re.findall(pattern,wareki)
    if len(date)>0:
        gengo = date[0][0]
        wareki_nen = kanji_to_number(date[0][1])
        wareki_tsuki = kanji_to_number(date[0][2])
        wareki_hi = kanji_to_number(date[0][3])

        gengo_nen = gengo_dict.get(gengo)
        if not gengo_nen:
            return 0
        if wareki_nen =='' or wareki_tsuki == '' or wareki_hi == '':
            return 0
        else:
            return (gengo_nen + int(wareki_nen)) * 10000 + int(wareki_tsuki) * 100 + int(wareki_hi)
        



In [4]:
all_law_list = pd.read_csv('./all_xml/all_law_list.csv')
# 未施行のものはいったん無視する
all_law_list = all_law_list[all_law_list['未施行'].isna()]
# 法令番号をキーにできるようにする
all_law_list.set_index('法令番号',inplace=True)

In [5]:
# 日付に関する項目を和暦から西暦にする（ソートするため）
all_law_list['公布日'] = all_law_list['公布日'].apply(lambda x:wareki_to_seireki8(x))
all_law_list['改正法令公布日'] = all_law_list['改正法令公布日'].apply(lambda x:wareki_to_seireki8(x))
all_law_list['施行日'] = all_law_list['施行日'].apply(lambda x:wareki_to_seireki8(x))

In [6]:
# 各法令の中で、施行日が一番新しいデータに絞り込む
all_law_list = all_law_list.sort_values(['法令番号','施行日'],ascending=[True,False]).groupby('法令番号').head(1)

In [7]:
# 具体的にデータを確認
all_law_list.head(5)

,法令種別,法令名,法令名読み,旧法令名,公布日,改正法令名,改正法令番号,改正法令公布日,施行日,施行日備考,法令ID,本文URL,未施行
法令番号,,,,,,,,,,,,,
令和三年カジノ管理委員会・法務省令第一号,府省令,特定資金移動履行保証金及び特定資金受入保証金に関する規則,とくていしきんいどうりこうほしょうきんおよびとくていしきんうけいれほしょうきんにかんするきそく,NaN,20210716,NaN,NaN,20210716,20210719,NaN,503M62000010001,https://laws.e-gov.go.jp/law/503M62000010001/2...,NaN
令和三年カジノ管理委員会規則第一号,規則,カジノ管理委員会関係特定複合観光施設区域整備法施行規則,かじのかんりいいんかいかんけいとくていふくごうかんこうしせつくいきせいびほうしこうきそく,NaN,20210716,カジノ管理委員会関係特定複合観光施設区域整備法施行規則の一部を改正する規則,令和六年カジノ管理委員会規則第二号,20240325,20240401,NaN,503M62000000001,https://laws.e-gov.go.jp/law/503M62000000001/2...,NaN
令和三年カジノ管理委員会規則第三号,規則,カジノ管理委員会の所管する法令に係る情報通信技術を活用した行政の推進等に関する法律施行規則,かじのかんりいいんかいのしょかんするほうれいにかかるじょうほうつうしんぎじゅつをかつようした...,NaN,20210716,カジノ管理委員会の所管する法令に係る情報通信技術を活用した行政の推進等に関する法律施行規則の...,令和六年カジノ管理委員会規則第一号,20240301,20240301,NaN,503M62000000003,https://laws.e-gov.go.jp/law/503M62000000003/2...,NaN
令和三年カジノ管理委員会規則第二号,規則,特定複合観光施設区域整備法関係手数料規則,とくていふくごうかんこうしせつくいきせいびほうかんけいてすうりょうきそく,NaN,20210716,特定複合観光施設区域整備法関係手数料規則の一部を改正する規則,令和四年カジノ管理委員会規則第三号,20221102,20221102,NaN,503M62000000002,https://laws.e-gov.go.jp/law/503M62000000002/2...,NaN
令和三年デジタル庁令第一号,府省令,デジタル庁組織規則,でじたるちょうそしききそく,NaN,20210901,デジタル庁組織規則の一部を改正する庁令,令和六年デジタル庁令第四号,20240329,20240401,NaN,503M60004000001,https://laws.e-gov.go.jp/law/503M60004000001/2...,NaN


In [8]:
# 法令番号をキー、法令名を値にした辞書を作成
law_dict = all_law_list['法令名'].to_dict()

In [9]:
# XMLの「Sentence」から法令の条数を導出する関数
def getarticle(sentence):
    provision_sentence = sentence.findParent('MainProvision')
    if provision_sentence:
        provision_num = 'MainProvision'
    else:
        provision_sentence = sentence.findParent('SupplProvision')
        if provision_sentence and provision_sentence.get_attribute_list('AmendLawNum')[0]:
            provision_num = provision_sentence.get_attribute_list('AmendLawNum')[0]
        else:
            provision_num = 'SupplProvision'
    article_sentence = sentence.findParent('Article')
    if article_sentence:
        article_num = article_sentence.get_attribute_list('Num')[0]
    else:
        article_num = '0'
    paragraph_sentence = sentence.findParent('Paragraph')
    if paragraph_sentence:
        paragraph_num = paragraph_sentence.get_attribute_list('Num')[0]
    else:
        paragraph_num = '1'
    item_sentence = sentence.findParent('Item')
    if item_sentence:
        item_num = item_sentence.get_attribute_list('Num')[0]
    else:
        item_num = '0'
    return provision_num, article_num, paragraph_num, item_num, sentence.get_text()

In [10]:
# 関連法令を探してデータに追加する関数
def refer_list(soup, law_dict, law_id):
    return_list = []
    search_results = []
    synonym = {}

    search_results = []
    synonym = {}
    if not soup:
        return
    
    law_text = soup.decode_contents()

    # 正規表現パターン
    regex = re.compile(r'(?<=（)((?:令和|平成|昭和|大正|明治)[元一二三四五六七八九十]+年(?:法律|政令|(?:[^）]*?省令)|内閣府令)第[一二三四五六七八九十百千万]+号)(?:。以下「([^）]*?)」という。)?(?=）)')
    
    # 正規表現にマッチする部分を抽出
    matches = regex.findall(law_text)
    
    for match in matches:
        if match[0]:
            search_results.append(match[0])
            if match[1]:
                synonym[match[0]] = match[1]
    
    # 正規表現パターン（略称名を別途定義する場合に対応）
    for law_num in search_results:
        regex_synonym = re.compile((f"{law_dict[law_num]}" if law_num in law_dict else law_num) + r"（以下「([^「]*?)」という。）")
        # 正規表現にマッチする部分を抽出
        matches = regex_synonym.findall(law_text)
        for match in matches:
            if match[0]:
                synonym[law_num] = match[0]
    
    # searchResultsをソート
    # 法律名の中には「貸付信託法」「信託法」のように短い法律名が長い法律名の一部になっているケースがあるため、誤った参照をしないように長さ順に並べ替えておく
    search_results = sorted(search_results, key=lambda x: len(law_dict.get(x, x)), reverse=True)
    for law_num in search_results:
        # 他法令の参照箇所を探す正規表現
        law_regex = re.compile(
            '(?:' + (f"{law_dict[law_num]}" if law_num in law_dict else law_num) + (f"|{synonym[law_num]}" if law_num in synonym else "") + ')' +
            r"(?:（(?:" + law_num + r")?。?(?:以下「[^「]*?」という。)?）)?第([一二三四五六七八九十百千万]+)条(?:の([一二三四五六七八九十百千万]+))?(?:第([一二三四五六七八九十百千万]+)項)?(?:第([一二三四五六七八九十百千万]+)号)?"
        )

        law_elements = soup.find_all("Sentence")

        for e in law_elements:
            original_html = e.decode_contents()
            results = law_regex.findall(original_html)
            for result in results:
                ref_provision_num, ref_article_num, ref_paragraph_num, ref_item_num, ref_text = getarticle(e)
                referred_provision_num = 'MainProvision'
                referred_article_num = kanji_to_number(result[0]) + ('_' + kanji_to_number(result[1]) if kanji_to_number(result[1]) !='' else '')
                referred_paragraph_num = kanji_to_number(result[2])
                referred_item_num = kanji_to_number(result[3])
                return_list.append({
                    'ref':{
                        'lawNum':law_id,
                        'lawArticle':{
                            'provision':ref_provision_num,
                            'article':ref_article_num,
                            'paragraph':ref_paragraph_num,
                            'item':ref_item_num,
                        },
                        'text':ref_text
                    },
                    'referred':{
                        'lawNum':law_num,
                        'lawArticle':{
                            'provision':referred_provision_num,
                            'article':referred_article_num,
                            'paragraph':referred_paragraph_num,
                            'item':referred_item_num,
                        }
                    },
                })
    return return_list


all_law_list.csvに掲載されている「本文URL」は、「https://laws.e-gov.go.jp/law/」に続けて【法令ID】/【適用日】_【法令ID】となっている。\
XMLファイルは「all_xml」フォルダ内の「【法令ID】_【適用日】_【法令ID】」に格納されており、ファイル名も「【法令ID】_【適用日】_【法令ID】.xml」となっている。\
※以下2件が例外になっているがレアケースとして無視する
- 205M10000200030_19431115_000000000000000 : 大正五年農商務省令第三十号（農商省主管歳入証券納付ニ関スル件）
- 332M50000040015_20200907_502M60000040021 : 租税特別措置法施行規則

In [11]:
reference_list = []
for id,r in tqdm(all_law_list.iterrows(),total=len(all_law_list)):

    url = r['本文URL']
    path = url.replace('https://laws.e-gov.go.jp/law/','').replace('/','_')
    if os.path.exists('./all_xml/' + path + '/' + path + '.xml'):
        file = './all_xml/' + path + '/' + path + '.xml'
        with open(file, mode='r', encoding='utf-8') as f:
            soup = bs(f, features='xml')
            reference_list += refer_list(soup, law_dict, id)

  0%|          | 0/8836 [00:00<?, ?it/s]

In [12]:
reference_df = pd.json_normalize(reference_list,)

In [13]:
reference_df.drop_duplicates(inplace=True)

In [14]:
reference_df.loc[reference_df['referred.lawArticle.article']=='','referred.lawArticle.article'] = '0'
reference_df.loc[reference_df['referred.lawArticle.paragraph']=='','referred.lawArticle.paragraph'] = '1'
reference_df.loc[reference_df['referred.lawArticle.item']=='','referred.lawArticle.item'] = '0'

In [15]:
reference_df.to_csv('reference.csv',index=False)

In [16]:
referred_lawNum = reference_df['referred.lawNum'].unique()

In [17]:
len(referred_lawNum)

4606

In [18]:
referred_lawNum = [lawNum for lawNum in referred_lawNum if law_dict.get(lawNum)]
len(referred_lawNum)

3812